In [3]:
import hadoop_lib_utils
HI_SYSTEMS = hadoop_lib_utils.get_dsxhi_info(showSummary=True)

Available Hadoop systems: 

            systemName LIVYSPARK  LIVYSPARK2       imageId
0   Chell Edge (shad1)            livyspark2  jupyter-py36
1      ak-cdh63-edge-1            livyspark2              
2        ak-cdhv711-ha            livyspark2  jupyter-py36
3   ak-mdd-cdh63-edge1            livyspark2  jupyter-py36
4      cdh513-outsider            livyspark2              
5         cdhv621-edge            livyspark2  jupyter-py36
6         cdhv714-edge            livyspark2              
7         hdp31-edge-1            livyspark2              
8      mjoucdh516-edge            livyspark2              
9               rated3            livyspark2  jupyter-py36
10            yccdh611            livyspark2              
11            yccdh711            livyspark2              


In [36]:
myConfig={
 "queue": "default",
 "driverMemory": "1G",
 "numExecutors": 1
}

HI_CONFIG = hadoop_lib_utils.setup_livy_sparkmagic(
                          system="yccdh711", 
                          livy="livyspark2",
                          addlConfig=myConfig)
print (HI_CONFIG)

# (Re-)load spark magic to apply the new configs.
%reload_ext sparkmagic.magics

sparkmagic has been configured to use https://internal-nginx-svc:12443/ibm-dsxhi-yccdh711/livy2/v1 
success configuring sparkmagic livy.
{'LIVY': 'https://internal-nginx-svc:12443/ibm-dsxhi-yccdh711/livy2/v1', 'SYSTEM': 'yccdh711', 'WEBHDFS': 'https://internal-nginx-svc:12443/ibm-dsxhi-yccdh711/webhdfs/v1', 'IMAGE': None}


In [37]:
%%time
session_name = 'test-sesh'
livy_endpoint = HI_CONFIG['LIVY']
webhdfs_endpoint = HI_CONFIG['WEBHDFS']
%spark add -s $session_name -l python -k -u $livy_endpoint

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
63,application_1609070229512_0079,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.
CPU times: user 227 ms, sys: 36.7 ms, total: 263 ms
Wall time: 27.3 s


In [34]:
%spark info

Info for running Spark:
    Sessions:
        Name: test-sesh	Session id: 59	YARN id: application_1609070229512_0075	Kind: pyspark	State: idle
	Spark UI: https://internal-nginx-svc:12443/ibm-dsxhi-yccdh711/gateway/wsl301-cpd-wsl301/yarn/proxy/application_1609070229512_0075
	Driver Log: https://internal-nginx-svc:12443/ibm-dsxhi-yccdh711/gateway/wsl301-cpd-wsl301/yarn/nodemanager/node/containerlogs/container_e07_1609070229512_0075_01_000001/user1?scheme=https&host=yccdh713.fyre.ibm.com&port=8044
    Session configs:
        {'queue': 'default', 'driverMemory': '1G', 'numExecutors': 1, 'proxyUser': 'user1', 'conf': {'spark.yarn.appMasterEnv.HI_UTILS_PATH': '/user/dsxhi/environments/pythonAddons/hi_core_utils.zip'}}



In [4]:
import os, json, sys, time, datetime, socket
import logging, requests
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %I:%M:%S %p',level=logging.INFO)
headers= {'Authorization': 'Bearer {}'.format(os.environ['USER_ACCESS_TOKEN'])}

def get_full_job_logs(batchApplicationId):
    '''
        Retrieve the FULL application stdout/stderr logs once an application has completed in YARN.
        
        :param: applicationId
    '''
    DSXHI_REST_APP_URL = "{}dsxhi/v1/job".format(HI_CONFIG['LIVY'].split('livy2')[0])
    print(DSXHI_REST_APP_URL)

    try:
        logging.info(">> Retrieving Yarn Application stdout/stderr logs")
        sesssion_req = requests.get("{}/log?jobId={}&onlyJobOutput=true".format(DSXHI_REST_APP_URL,batchApplicationId),
                            headers=headers,
                            verify=False,
                            timeout=60)
        if sesssion_req.status_code == 200:
            # API Returns "bytes"
            print(sesssion_req.content.decode())
        else:
            print(sesssion_req.status_code)
            print(sesssion_req.content.decode())
            sys.exit(1)
    except Exception as ee:
        logging.error("Error retrieving yarn logs: %s",ee)
        sys.exit(1)

In [ ]:
# Will be empty while job is running
# Updatee the app_id to value returned from Livy above!

get_full_job_logs('application_1609070229512_0003')